In [ ]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt


In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
def f_wb(x, w, b):
    return sigmoid(np.dot(w, x) + b)

In [ ]:
def loss(y, y_hat):
    return -y * np.log(y_hat) - (1 - y) * np.log(1 - y_hat)

In [ ]:
def compute_cost_logistic(X, Y, w, b):
    m = X.shape[0] 
    cost = 0.0
    for i in range(m):
        cost += loss(Y[i], f_wb(X[i], w, b))
        
    return cost / m

In [ ]:
X_train = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])  #(m,n)
y_train = np.array([0, 0, 0, 1, 1, 1])                                           #(m,)

w_tmp = np.array([1,1])
b_tmp = -3
print(compute_cost_logistic(X_train, y_train, w_tmp, b_tmp))

# Plot the data
fig, ax = plt.subplots(1, 1, figsize = (7, 7))
x0 = np.arange(0, 6)
x1 = 3 - x0
for i in range(X_train.shape[0]):
    ax.plot(X_train[i][0], X_train[i][1], marker = 'o' if y_train[i] == 0 else 'x', color = 'r' if y_train[i] == 0 else 'b', label = '0' if y_train[i] == 0 else '1')

ax.plot(x0, x1, label = 'w0*x0 + w1*x1 + b = 0', color="magenta")
ax.set_xlabel("$x_0$", fontsize = 12)
ax.set_ylabel("$x_1$", fontsize = 12)
plt.title("Dicision Boundary")
plt.legend(loc='upper right')
plt.show()

In [ ]:
w_array1 = np.array([1,1])
b_1 = -3
w_array2 = np.array([1,1])
b_2 = -4

print("Cost for b = -3 : ", compute_cost_logistic(X_train, y_train, w_array1, b_1))
print("Cost for b = -4 : ", compute_cost_logistic(X_train, y_train, w_array2, b_2))